In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
api_key = "api-key"

In [2]:
list1 = pd.read_csv('classified_data/list1_query_classified.csv')
list2 = pd.read_csv('classified_data/list2_query_classified.csv')

list1_m = pd.read_csv('deepdao_metrics/list1_deepdao_metrics.csv')
list2_m = pd.read_csv('deepdao_metrics/list2_deepdao_metrics.csv')

In [3]:
def dao_participation_metrics(address):

    endpoint_url = f"https://api.deepdao.io/v0.1/people/dao_participation_score/{address}"
    headers = {"x-api-key":api_key}

    response = requests.get(endpoint_url, headers=headers)

    if response.status_code == 200:
        data = response.json()['data']
        return data
        
    else:
        print(f"Error: {response.status_code}")

def is_active_in_daos(address):

    endpoint_url = f"    https://api.deepdao.io/v0.1/people/is_active_in_daos/{address}"
    headers = {"x-api-key":api_key}

    response = requests.get(endpoint_url, headers=headers)

    if response.status_code == 200:
        data = response.json()['data']
        return data
        
    else:
        print(f"Error: {response.status_code}")

In [4]:
df = [['address','score', 'rank', 'relativeScore', 'daos', 'proposals', 'votes', 'isSmartContract','isActive_in_Daos','list_no']]

def testing(dataframe):
    count = 0
    for address in dataframe['address'][642:]:
        dpm = dao_participation_metrics(address)
        ad = is_active_in_daos(address)

        row = list(dpm.values())
        row.insert(0,address)
        row.append(next(iter(ad.values())))
        row.append(1)
        df.append(row)
        print(count)
        count += 1

In [5]:
#testing(list2)
#list2_full = pd.DataFrame(df[1:], columns = df[0])
#list2_full.to_csv('deepdao_metrics/list2_deepdao_metrics.csv', index=False)

In [6]:
list1.head(1)

,address,balance,TDR,Wallet_Age(Days),From_Count,To_Count,Predicted_Holding_days
0,0x5e23d08324f017d5425e59a2782c9ae27ace0958,0.0,0.0,0,0,0,Over 4 weeks


In [7]:
list1_full = pd.merge(list1,list1_m,on='address')
list2_full = pd.merge(list2,list2_m,on='address')

In [8]:
list2_full.head(3)

,address,balance,TDR,Wallet_Age(Days),From_Count,To_Count,Predicted_Holding_days,score,rank,relativeScore,daos,proposals,votes,isSmartContract,isActive_in_Daos,list_no
0,0xd9293636ee3aa663fc563adb0912b0705dafb62c,0.02663,0.568,684,47,53,Less than a week,325.84,2,100.0,128,12,3860,No,True,1
1,0x8eef779818afa953b0652e45438423ebe089f55a,0.08224,0.422,602,33,67,Less than a week,154.28,9,100.0,93,18,1403,No,True,1
2,0xe274ae86ff13c1bbaacd3662ec5fd7279a18e28f,0.01211,0.093,474,10,28,Over 4 weeks,56.82,372,100.0,189,0,501,No,True,1


In [9]:
list1_full.drop('list_no',axis=1,inplace=True)
list2_full.drop('list_no',axis=1,inplace=True)

In [14]:
main = list1_full.loc[(list1_full['Predicted_Holding_days']=='Over 4 weeks')|(list1_full['Predicted_Holding_days']=='3-4 weeks')]

In [21]:
main['votes'].value_counts()

0      16
485     9
356     8
490     8
482     7
       ..
545     1
358     1
453     1
431     1
244     1
Name: votes, Length: 397, dtype: int64

In [15]:
main.describe()

,balance,TDR,Wallet_Age(Days),From_Count,To_Count,score,rank,relativeScore,daos,proposals,votes
count,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000
mean,0.222304,0.274753,597.330909,21.540606,36.660606,57.811248,516.352727,99.994315,77.753939,13.054545,450.778182
std,1.217565,0.813403,495.070480,18.301178,26.038363,22.869556,296.922766,0.004956,77.733886,77.959525,193.641015
min,0.000000,0.000000,0.000000,0.000000,0.000000,37.110000,1.000000,99.990000,1.000000,0.000000,0.000000
25%,0.002650,0.048000,284.000000,4.000000,13.000000,44.520000,266.000000,99.990000,33.000000,0.000000,342.000000
50%,0.020780,0.134000,516.000000,19.000000,39.000000,52.260000,511.000000,99.990000,47.000000,0.000000,424.000000
75%,0.090020,0.260000,760.000000,38.000000,59.000000,60.270000,762.000000,100.000000,71.000000,1.000000,501.000000
max,22.077210,14.286000,2282.000000,90.000000,100.000000,361.630000,1199.000000,100.000000,407.000000,1595.000000,1453.000000
